# Tutorial - Soil profile objects in Groundhog

A soil profile is a table with several layers where the bottom depth of the previous layer corresponds to the top depth of the next layer.

Because a soil profile is a dataframe with additional functionality, the ```SoilProfile``` class inherits from the ```DataFrame``` class.

Additional functionality is implemented to enable all common soil profile manipulations such:
   - Retrieving minimum and maximum depth;
   - Changing depth coordinate signs;
   - Changing the mudline level;
   - Retrieving the soil parameters available in the dataframe;
   - Mapping the soil parameters to a grid;
   - Plotting the soil profile;
   
This tutorial demonstrates this functionality.

In [ ]:
import numpy as np
from groundhog.general import soilprofile as sp
from groundhog.__version__ import __version__
__version__

```groundhog``` uses Plotly as the plotting backend. Please note that you may still need to install Plotly in your Python environment.

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot, download_plotlyjs
init_notebook_mode()

## 1. Soil profile creation

There are a couple of standard scenarios for creating soil profiles:

   - Soil profile definition based on a Python dictionary;
   - Soil profile reading from Excel file
  
These two scenarios are demonstrated here.

### 1.1. Soil profile creation from Excel file

When a soil profile is created from Excel, the layer are encoded as rows and soil parameters names are put on the first row. The coordinate of the top of the next layer should always correspond to the coordinate of the bottom of the previous layer. There is one all-important convention for soil parameters:

   - Numerical soil parameters have units between square brackets (e.g. ```qc [MPa]```)
   - Numerical soil parameters can have linear variations using the ```from``` and ```to``` words in the title (e.g. ```qc from [MPa]``` and ```qc to [MPa]```)
   - String soil parameters are specified without units, square brackets should not be used in the title
   
The user can use several names for the depth from and depth to columns. ```Depth from [m]``` and ```Depth to [m]``` are used by default but different names and units can be used by specifying the ```depth_key``` and ```unit``` keyword arguments.

As an example a file with depth (z) specified in imperial units can be read.

In [ ]:
profile_1 = sp.read_excel("Data/soilprofile_basic.xlsx")
profile_1

In [ ]:
profile_1.calculate_overburden()
profile_1[['Depth from [m]', 'Depth to [m]',
           'Vertical effective stress from [kPa]', 'Vertical effective stress to [kPa]',
           'Vertical total stress from [kPa]', 'Vertical total stress to [kPa]']]

### 1.2. Soil profile from dictionary

A soil profile can be directly specified in the notebook through a dictionary. The same profile can be loaded but in SI units. Note that ```Depth from [m]``` and ```Depth to [m]``` are required here.

In [ ]:
profile_2 = sp.SoilProfile({
    'Depth from [m]': [0, 1, 3, 4],
    'Depth to [m]': [1, 3, 4, 10],
    'Soil type': ['SAND', 'CLAY', 'SILT', 'SAND'],
    'Relative density': ['Loose', None, 'Medium dense', 'Dense'],
    'qc from [MPa]': [3, 1, 4, 40],
    'qc to [MPa]': [4, 1.5, 8, 50],
    'qt [MPa]': [3.5, 1.25, 6, 45],
    'Total unit weight [kN/m3]': [19, 18, 19, 20]
})
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]', 'qt [MPa]')),
    showlegends=((False,), (True, True)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $', r'$ q_c, \ q_t \ \text{[MPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60)),
    zrange=(10, 0),
    fillcolordict={'SAND': 'yellow', 'CLAY': 'brown', 'SILT': 'green'})

## 2. Retrieving information from ```SoilProfile``` objects

```SoilProfile``` objects have properties which allow the user to quickly assess the contents of the soil profile.

### 2.1. Top and bottom depth

The minimum and maximum depth of the soil profile can be retrieved using ```min_depth``` and ```max_depth``` attributes.

In [ ]:
profile_2.min_depth, profile_2.max_depth

### 2.2. Soil parameters

The ```SoilProfile``` objects has a method to retrieve the numerical and string soil parameters.

In [ ]:
profile_2.numerical_soil_parameters()

In [ ]:
profile_2.string_soil_parameters()

For the numerical soil parameters, the method ```check_linear_variation``` allows to check whether the parameter is constant in the layer or whether is has a linear variation. Linear variations are encoded in the soil profile by using the ```to``` and ```from``` column keys (e.g. ```qc from [MPa]``` and ```qc to [MPa] ```).

In [ ]:
for _param in profile_2.numerical_soil_parameters():
    if profile_2.check_linear_variation(_param):
        print("Parameter %s shows a linear variation" % _param)
    else:
        print("Parameter %s is constant in each layer" % _param)

## 3. Selection of soil parameters

The ```SoilProfile``` object has a method for automatic selection of design lines based on parameter values in the layer. This can be demonstrated using a couple of randomly selected value for the undrained shear strength.

In [ ]:
depths = np.linspace(1.1, 2.9, 25)
su_values = 20 + 20 * np.random.rand(25)

In [ ]:
profile_2.selection_soilparameter(
    parameter='Su [kPa]',
    depths=depths,
    values=su_values,
    rule='mean',
    linearvariation=True)
profile_2

The selected line can be plotted by adding a trace to a plot with a mini-log.

In [ ]:
su_trace = go.Scatter(x=su_values, y=depths, mode='markers', name='Su measurements')

fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]', 'qt [MPa]'), ('Su [kPa]',)),
    showlegends=((False,), (True, True), (False,)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60)),
    zrange=(10, 0),
    showfig=False)

fig.append_trace(su_trace, 1, 4)
iplot(fig)

## 4. Soil profile manipulations

A number of manipulations with soil profiles are possible using the ```SoilProfile``` class.

### 4.1. Shifting vs depth

The profile can be shifted vs depth using the ```shift_depths``` method. For example we can move the profile up by 5m. Note: Moving up requires a negative offset to be specified (depth axis is positive in the downward direction).

In [ ]:
profile_2.shift_depths(offset=-4)
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]', 'qt [MPa]'), ('Su [kPa]',)),
    showlegends=((False,), (True, True), (False,)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60)),
    zrange=(6, -4))

Each time the ```shift_depths``` method is applied, a further shift is applied, so be careful not to repeat code containing this method inadvertently.

### 4.2. Flipping the depth axis

In certain cases (e.g. when working with depths in mLAT), flipping of the depth axis is required. This can be done using the ```convert_depth_sign```.

In [ ]:
profile_2.convert_depth_sign()
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]', 'qt [MPa]'), ('Su [kPa]',)),
    showlegends=((False,), (True, True), (False,)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60)),
    zrange=(-6, 4))

This statement can also be repeated. Note that most other methods of the ```SoilProfile``` object expect depths increasing downward!

For the further demonstrations of the functionality, we will reset the depth reference:

In [ ]:
profile_2.convert_depth_sign()
profile_2.shift_depths(offset=4)
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]', 'qt [MPa]'), ('Su [kPa]',)),
    showlegends=((False,), (True, True), (False,)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60)),
    zrange=(10, 0))

### 4.3. Inserting a layer transition

Inserting a layer transition is easily achieved using the ```insert_layer_transition``` method.

In [ ]:
profile_2.insert_layer_transition(depth=8)
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]', 'qt [MPa]'), ('Su [kPa]',)),
    showlegends=((False,), (True, True), (False,)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60)),
    zrange=(10, 0))

### 4.4. Merging layers

Layers can be merged using their index (starting from 0 for the top layer). Note that the functionality still needs to be completed for layers with linearly varying properties. By default, the properties of the top layer are kept.

In [ ]:
profile_2.merge_layers(layer_ids=(3, 4))
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]', 'qt [MPa]'), ('Su [kPa]',)),
    showlegends=((False,), (True, True), (False,)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60)),
    zrange=(10, 0))

### 4.5. Removing soil parameters

A soil parameter can be removed using its name.

In [ ]:
profile_2.remove_parameter(parameter="qt [MPa]")
profile_2

### 4.6. Cutting a soil profile

A specific section of the soil profile can be ```cut_profile``` method. A deep copy of the soil profile is then returned which is a ```SoilProfile``` object in itself. The cutting process takes linearly varying parameters into consideration.

In [ ]:
profile_extract = profile_2.cut_profile(top_depth=0.5, bottom_depth=8)
fig = profile_extract.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]', ), ('Su [kPa]',)),
    showlegends=((False,), (False,), (False,)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60)),
    zrange=(8, 0.5))

### 4.7. Integration of a soil parameter vs depth

A soil parameter can be integrated over the depth and the resulting property can be added to the ```SoilProfile``` dataframe. This only works for soil parameters with a constant value in each layer and with properties specified in each layer (no NaN values). This can be demonstrated for the vertical effective stress, as integrated from the effective unit weight.

In [ ]:
profile_2.depth_integration(parameter='Total unit weight [kN/m3]', outputparameter='Vertical total stress [kPa]')
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]',), ('Su [kPa]',), ('Vertical total stress [kPa]',)),
    showlegends=((False,), (False,), (False,), (False,)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $',
             r'$ \sigma_{vo}^{\prime} \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60), (0, 200)),
    zrange=(10, 0))

Since calculation of overburden is a recurring task in geotechnical analyses, the method ```calculate_overburden``` is implemented to calculate hydrostatic water pressure, total and effective vertical stress with a single statement.

The water level can be adjusted. If a layer interface is not present at the location of the water level, an additional interface is created. The soil profile needs to contain a column with the total unit weight to allow the calculation to happen. In layers above the water level, the total unit weight is the dry unit weight and the effective unit weight is equal to this value. In the layers below the water table, the effective unit weight is obtained by subtracting the water unit weight (default 10kN/m$^3$) from the total unit weight.

In [ ]:
profile_2.calculate_overburden(waterlevel=2.5)

In [ ]:
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]',), ('Su [kPa]',),
                ('Vertical effective stress [kPa]', 'Vertical total stress [kPa]', 'Hydrostatic pressure [kPa]')),
    showlegends=((False,), (False,), (False,), (True, True, True)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $',
             r'$ \sigma_{vo}, \ \sigma_{vo}^{\prime}, \ u_0 \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60), (0, 200)),
    zrange=(10, 0))

## 5. Gridding functionality

The ```SoilProfile``` object can be mapped onto a grid. All that is required is a list or Numpy array with the depth coordinates of the grid. The method ```map_soilprofile``` returns a dataframe with the mapped soil parameters.

In [ ]:
grid = profile_2.map_soilprofile(
    nodalcoords=np.linspace(0, 10, 21))
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]',), ('Su [kPa]',), ('Vertical total stress [kPa]',)),
    showlegends=((False,), (False,), (False,), (False,)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $',
             r'$ \sigma_{vo}^{\prime} \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60), (0, 200)),
    zrange=(10, 0),
    showfig=False)

qc_grid_trace = go.Scatter(x=grid['qc [MPa]'], y=grid['z [m]'], mode='markers', name='Gridded qc',
                           marker=dict(size=8,color='red',line=dict(width=2,color='black')))
fig.append_trace(qc_grid_trace, 1, 3)
iplot(fig)

In [ ]:
grid.head()

## 6. Changing the depth scale

When converting from metric to imperial units and vice versa, depth scales need to be converted from m to ft.

This can be done using the ```.convert_depth_reference``` method. The new unit name and the conversion factor need to be specified:

In [ ]:
profile_2.convert_depth_reference(newunit='ft', multiplier=1/0.3048)

In [ ]:
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]',), ('Su [kPa]',),
                ('Vertical effective stress [kPa]', 'Vertical total stress [kPa]', 'Hydrostatic pressure [kPa]')),
    showlegends=((False,), (False,), (False,), (True, True, True)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $',
             r'$ \sigma_{vo}, \ \sigma_{vo}^{\prime}, \ u_0 \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[ft]} $',
    xranges=((15, 22), (0, 60), (0, 60), (0, 200)),
    zrange=(10 / 0.3948, 0))

The depth scale can be converted back to meters again:

In [ ]:
profile_2.convert_depth_reference(newunit='m', multiplier=0.3048)

In [ ]:
fig = profile_2.plot_profile(
    parameters=(('Total unit weight [kN/m3]',), ('qc [MPa]',), ('Su [kPa]',),
                ('Vertical effective stress [kPa]', 'Vertical total stress [kPa]', 'Hydrostatic pressure [kPa]')),
    showlegends=((False,), (False,), (False,), (True, True, True)),
    xtitles=(r'$ \gamma \ \text{[kN/m} ^3 \text{]} $',
             r'$ q_c, \ q_t \ \text{[MPa]} $',
             r'$ S_u \ \text{[kPa]} $',
             r'$ \sigma_{vo}, \ \sigma_{vo}^{\prime}, \ u_0 \ \text{[kPa]} $'),
    ztitle=r'$ z \ \text{[m]} $',
    xranges=((15, 22), (0, 60), (0, 60), (0, 200)),
    zrange=(10, 0))